### 1. Goal
To classify the headers of the Bill of Materials (BOM) table uploaded by users into the categories.

The Relationship Between Categories and BOM Headers

| Categories       | BOM Headers |
|-----------------|------------------------------------------------------------|
| **公司料號**    | Comp_item, 元件料號, serial number, 子件代碼, 物料型號 |
| **料號描述**    | Description, Size/Dimension, Comment, Designator, 材料規格, 物料描述 |
| **料號淨重**    | Net weight |
| **料號毛重**    | Gross weight |
| **淨毛重單位**  | 單位, unit, Net weight unit, Gross weight unit |

### 2. User Story
When users upload a new, unseen BOM table (e.g., `test_bom.xlsx`), the service needs to classify the headers into corresponding categories.

True Relationship Between `test_bom.xlsx` Headers and Pre-defined Categories

| Categories       | BOM Headers |
|-----------------|------------|
| **公司料號**    | 料號 |
| **料號描述**    | 規格, 備註 |
| **料號淨重**    | 淨重 |
| **料號毛重**    | 毛重 |
| **淨毛重單位**  | 重量單位 |

### 3. Classification examples with test_bom
#### 3.1 Matched categories with its headers:
<span style="color:yellow;">-For demostration, few of the BOM headers from test_bom are classified wrongly or missing.
```json
{
    '公司料號':['料號'],
    '料號描述':['規格', '備註','重量單位'],
    料號毛重:['毛重']
}
```

#### 3.2 Unmatched categories:
<span style="color:yellow;"> -Categories 料號淨重 and 淨毛重單位 are not matched. <u>This can be detected by setting rule</u>, for example minimum 1 matching headers should be found.
```json
{
    '料號毛重',
    '淨毛重單位'
}
```


#### 3.3 Matched categories with wrong headers:
<span style="color:yellow;"> -重量單位 is missclassified into 料號描述, correct classification is 料號淨重. <u>This should be detected by validation after classification</u>, for example validate the data type of column.
```json
{
    '料號描述':['重量單位']
}
```


#### 3.4 Unmatched bom headers:
<span style="color:yellow;"> -This can be either model fail to classify bom headers.
```json
{'淨重'}
```



pip or conda to install openpyxl

In [ ]:
import pandas as pd
import numpy as np
data_path="data/"
import os

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
import google.generativeai as genai


import json
import re

In [ ]:
bom1=pd.read_excel(data_path+"bom1.xlsx")
bom2=pd.read_excel(data_path+"bom2.xlsx")
test_bom=pd.read_excel(data_path+"test_bom.xlsx")
case1=pd.read_excel(data_path+"case1.xlsx")

In [ ]:
ground_truth={
    'Categories':['Company Part No.','Part Description','Part Net Weight','Part Gross Weight','Net/Gross Unit'],
    'BOM headers':[
        ['Company Part No.','公司料號','Comp_item','元件料號','serial number','子件代碼','物料型號'],
        ['Part Description','料號描述','Description', 'Size/Dimension', 'Comment', 'Designator','材料規格', '物料描述'],
        ['Part Net Weight','料號淨重','Net weight'],
        ['Part Gross Weight','料號毛重','Gross weight'],
        ['Net/Gross Unit','料號毛重','單位','unit', 'Net weight unit','Gross weight unit']
    ]
    }

reversed_ground_truth = {}
for category, headers in zip(ground_truth['Categories'], ground_truth['BOM headers']):
    for header in headers:
        reversed_ground_truth[header] = category
print(reversed_ground_truth)

In [369]:
train_df=pd.DataFrame({'BOM headers':reversed_ground_truth.keys(),'Categories':reversed_ground_truth.values()})
train_df['strip_BOM_headers']=train_df['BOM headers'].apply(lambda x: x.replace("_","").replace(" ",""))
train_df.head()

,BOM headers,Categories,strip_BOM_headers
0,Company Part No.,Company Part No.,CompanyPartNo.
1,公司料號,Company Part No.,公司料號
2,Comp_item,Company Part No.,Compitem
3,元件料號,Company Part No.,元件料號
4,serial number,Company Part No.,serialnumber


In [ ]:
helper = ChatGoogleGenerativeAI(model="gemini-2.0-flash",temperature=1,max_tokens=None,timeout=None,)

In [343]:
promt="""
Your goal is to help data engineer to understand inventory data schema header
Output english descriptions for all inventory data schema headers in the list below into json format.
For example，
Given: ['Company Part No.', '公司料號']
Return: 
```json
    "Company Part No.":"Unique identifier assigned to a part by the company. This is the internal part number used for tracking and identification within the organization.",
    "公司料號":"Unique identifier assigned to a part by the company (in Chinese). This is the internal part number used for tracking and identification within the organization."
```
Begin now:
"""

In [350]:
header_description=helper.invoke(promt+str(list(train_df['BOM headers'])))
print(header_description.content)

```json
{
    "Company Part No.":"Unique identifier assigned to a part by the company. This is the internal part number used for tracking and identification within the organization.",
    "公司料號":"Unique identifier assigned to a part by the company (in Chinese). This is the internal part number used for tracking and identification within the organization.",
    "Comp_item":"Abbreviated form of 'Company Item'. Likely refers to the company's part number or item code.",
    "元件料號":"Part number of a component (in Chinese). This refers to the identification code of a specific electronic component or sub-assembly.",
    "serial number":"A unique identifier assigned to a specific instance of a part, allowing for individual tracking.",
    "子件代碼":"Sub-component code (in Chinese). This is the identification code for a sub-component or sub-assembly within a larger part.",
    "物料型號":"Material model/type (in Chinese). This refers to the specific model or type of material used for the part.",
    "

In [351]:
description=re.sub(r"^```json\n|```$", "", header_description.content, flags=re.MULTILINE)
header_description_json=json.loads(description)
header_description_df=pd.DataFrame({"BOM headers":header_description_json.keys(),"column_description":header_description_json.values()})

In [370]:
train_df=train_df.merge(header_description_df,how='left',on='BOM headers')
train_df['column_description']=train_df['BOM headers']+": "+train_df['column_description']
train_df.head(3)

,BOM headers,Categories,strip_BOM_headers,column_description
0,Company Part No.,Company Part No.,CompanyPartNo.,Company Part No.: Unique identifier assigned t...
1,公司料號,Company Part No.,公司料號,公司料號: Unique identifier assigned to a part by ...
2,Comp_item,Company Part No.,Compitem,Comp_item: Abbreviated form of 'Company Item'....


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

https://python.langchain.com/api_reference/huggingface/embeddings/langchain_huggingface.embeddings.huggingface.HuggingFaceEmbeddings.html

In [371]:
bom_headers_embedding_list=[]
for c in train_df['BOM headers']:
    embedding = embeddings_model.embed_query(c)
    bom_headers_embedding_list.append(embedding)
print(bom_headers_embedding_list)

[[6.642220978392288e-05, -0.010337683372199535, -0.01148230955004692, -0.03415466472506523, 0.005939788185060024, -0.0003789965121541172, 0.004023274406790733, 0.052121374756097794, 0.04695210978388786, -0.02690179832279682, 0.042652033269405365, 0.006332517601549625, -0.05020089074969292, -0.03272632509469986, -0.015712684020400047, -0.026301229372620583, -0.018420176580548286, 0.008542709983885288, -0.004355146083980799, 0.0021711040753871202, 0.026408711448311806, -0.012764856219291687, -0.042218755930662155, -0.0038842298090457916, 0.0007194860954768956, -0.04637666419148445, -0.04588543251156807, -0.023558130487799644, -0.01675584726035595, -0.03137725964188576, 0.021346934139728546, 0.009896727278828621, -0.04399774223566055, -0.012245206162333488, -0.03025130368769169, 0.02732754312455654, 0.052898697555065155, 0.04571720212697983, -0.03843925520777702, 0.02778533287346363, -0.02797052264213562, 0.04984840378165245, 0.008450601249933243, -0.07426391541957855, -0.0019508205587044

In [372]:
descrription_embedding_list=[]
for c in train_df['column_description']:
    embedding = embeddings_model.embed_query(c)
    descrription_embedding_list.append(embedding)
print(descrription_embedding_list)

[[0.016675466671586037, -0.009528583846986294, -0.008120553568005562, -0.028050528839230537, 0.0229045283049345, -0.016598500311374664, 0.0058652726002037525, 0.04451722279191017, 0.035165827721357346, -0.010867471806704998, 0.03590379282832146, -0.00320032169111073, -0.045243583619594574, 0.008371581323444843, -0.019822435453534126, 0.003459571860730648, -0.010887105017900467, 0.02773677185177803, -0.01287719514220953, -0.013140516355633736, 0.04750397056341171, -0.039559535682201385, -0.054078951478004456, -0.005554836243391037, -0.0075404345989227295, -0.03791608288884163, -0.04628010094165802, 0.016466159373521805, -0.03082374483346939, -0.03976058214902878, -0.007065122947096825, 0.02547372505068779, -0.03130558133125305, -0.01576656475663185, -0.01909109205007553, 0.025598082691431046, 0.04638256877660751, 0.03415244072675705, -0.02084357477724552, 0.03187158331274986, -0.003253633389249444, 0.04902927204966545, 0.018041284754872322, -0.061114586889743805, 0.0020678711589425802, 

In [373]:
train_df['BOM_headers_embedding']=bom_headers_embedding_list
train_df['BOM_headers_relevant_score']=0
train_df['description_embedding']=descrription_embedding_list
train_df['description_relevant_score']=0
train_df.to_parquet("embeded_train_df.gzip")
train_df.head(3)

,BOM headers,Categories,strip_BOM_headers,column_description,BOM_headers_embedding,BOM_headers_relevant_score,description_embedding,description_relevant_score
0,Company Part No.,Company Part No.,CompanyPartNo.,Company Part No.: Unique identifier assigned t...,"[6.642220978392288e-05, -0.010337683372199535,...",0,"[0.016675466671586037, -0.009528583846986294, ...",0
1,公司料號,Company Part No.,公司料號,公司料號: Unique identifier assigned to a part by ...,"[0.01681557111442089, 0.01526159979403019, -0....",0,"[0.022944552823901176, -0.0008206613129004836,...",0
2,Comp_item,Company Part No.,Compitem,Comp_item: Abbreviated form of 'Company Item'....,"[0.010160735808312893, 0.025978170335292816, -...",0,"[0.008331971243023872, -0.00643406854942441, -...",0


In [ ]:
company_name,product_name,company_id, product_id
公司料號,公司商號，产品编號  
0.9 料號 毛重 重量單位 規格 備註 淨重 毛重

gender 性别
				


In [405]:
list(bom1.columns)

['Comp_item',
 'Description',
 'Size/Dimension',
 'Net weight',
 'Gross weight',
 'unit']

In [406]:
list(bom2.columns)

['元件料號', '物料描述', '材料規格', 'Net weight', 'Gross weight', '單位']

In [407]:
list(test_bom.columns)

['料號', '規格', '備註', '淨重', '毛重', '重量單位']

In [ ]:
input_list=+()+

In [408]:
train_df=pd.read_parquet("embeded_train_df.gzip")
input_prompt_list=[]
result_df=pd.DataFrame()
for input_header in list(test_bom.columns):
    promt="""
    Your goal is to help data engineer to understand the column header of inventory database.
    Provide column header description, must use the same language type as the given column header below.
    Strip off markdown formatting.
    For example: 
    Comp_item: The specific individual component or item that makes up part of the higher-level assembly or product being described in this inventory record
    公司料號: Unique identifier assigned to a part by the company (in Chinese). This is the internal part number used for tracking and identification within the organization.
    Explain column header below:
    """
    input_header_description=helper.invoke(promt+input_header)
    input_prompt_list.append(input_header_description.content)

    header_threshold=0.8
    description_threshold=0.8
    ranking=1
    header_embedding = embeddings_model.embed_query(input_header)   
    description_embedding = embeddings_model.embed_query(input_header_description.content)
    train_df['BOM_headers_relevant_score'] = np.dot(np.stack(train_df['BOM_headers_embedding']), header_embedding)
    train_df['description_relevant_score'] = np.dot(np.stack(train_df['description_embedding']), description_embedding)
    #bom_header_top_relevant=train_df.loc[(train_df['BOM_headers_relevant_score']>header_threshold)].sort_values('BOM_headers_relevant_score',ascending=False).head(ranking)
    description_top_relevant=train_df.loc[(train_df['description_relevant_score']>description_threshold)].sort_values('description_relevant_score',ascending=False).head(ranking)
    result_df=pd.concat([result_df,description_top_relevant[['BOM headers','column_description','description_relevant_score','BOM_headers_relevant_score','Categories']]])

In [409]:
result_df

,BOM headers,column_description,description_relevant_score,BOM_headers_relevant_score,Categories
1,公司料號,公司料號: Unique identifier assigned to a part by ...,0.960454,0.944468,Company Part No.
13,材料規格,材料規格: Material specifications (in Chinese). De...,0.924011,0.939752,Part Description
11,Comment,Comment: Any additional notes or comments rega...,0.925092,0.815895,Part Description
16,料號淨重,料號淨重: Net weight of the part number (in Chines...,0.935836,0.928177,Part Net Weight
20,Gross weight,Gross weight: The weight of the part including...,0.951193,0.821523,Part Gross Weight
22,單位,單位: Unit (in Chinese). The unit of measurement...,0.910669,0.931832,Net/Gross Unit


In [394]:
input_header='性别'
promt="""
Your goal is to help data engineer to understand the column header of inventory database.
Provide column header description, must use the same language type as the given column header below.
Strip off markdown formatting.
For example: 
Comp_item: The specific individual component or item that makes up part of the higher-level assembly or product being described in this inventory record
公司料號: Unique identifier assigned to a part by the company (in Chinese). This is the internal part number used for tracking and identification within the organization.
Explain column header below:
"""
input_header_description=helper.invoke(promt+input_header)
print(input_header_description.content)

train_df=pd.read_parquet("embeded_train_df.gzip")
header_threshold=0.8
description_threshold=0.80
ranking=5
header_embedding = embeddings_model.embed_query(input_header)   
description_embedding = embeddings_model.embed_query(input_header_description.content)
train_df['BOM_headers_relevant_score'] = np.dot(np.stack(train_df['BOM_headers_embedding']), header_embedding)
train_df['description_relevant_score'] = np.dot(np.stack(train_df['description_embedding']), description_embedding)
#bom_header_top_relevant=train_df.loc[(train_df['BOM_headers_relevant_score']>header_threshold)].sort_values('BOM_headers_relevant_score',ascending=False).head(ranking)
description_top_relevant=train_df.loc[(train_df['description_relevant_score']>description_threshold)].sort_values('description_relevant_score',ascending=False).head(ranking)

性别: The gender of the individual, typically categorized as male or female.


In [395]:
description_top_relevant[['BOM headers','column_description','description_relevant_score','BOM_headers_relevant_score','Categories']]

,BOM headers,column_description,description_relevant_score,BOM_headers_relevant_score,Categories
9,Description,Description: A general description of the item...,0.855153,0.760619,Part Description
23,unit,unit: The unit of measurement used for a quant...,0.852597,0.773680,Net/Gross Unit
22,單位,單位: Unit (in Chinese). The unit of measurement...,0.852168,0.835630,Net/Gross Unit
4,serial number,serial number: A unique identifier assigned to...,0.834247,0.775048,Company Part No.
14,物料描述,物料描述: Material description (in Chinese). A tex...,0.828737,0.843130,Part Description


In [ ]:
case0=train_df[['BOM headers','Categories']].sample(10)
case0=case0.set_index('BOM headers').transpose()
case0

In [ ]:
#find exact match in ground truth
def L1_match(bom_df):
    category_list=[]
    result_list=[]
    for c in bom_df.columns:
        c=c.replace("_","").replace(" ","")
        if c in list(train_df['strip_BOM_headers']):
            category_list.append(train_df[train_df['strip_BOM_headers']==c].iloc[0]['Categories'])
            result_list.append(1)
        else:
            category_list.append(None)
            result_list.append(0)
    result_df=pd.DataFrame({'BOM_headers':list(bom_df.columns),'result':result_list,'Categories':category_list})
    return result_df

In [ ]:
L1_match(case0)

In [ ]:
L1_match(case1)

In [ ]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define BOM category mappings
CATEGORY_MAP = {
    "Company Part No.": ["Company Part No.","公司料號","Comp_item", "元件料號", "serial number", "子件代碼", "物料型號"],
    "Part Description": ["Part Description","料號描述","Description", "Size/Dimension", "Comment", "Designator", "材料規格", "物料描述"],
    "Part Net Weight": ["Part Net Weight","料號淨重","Net weight"],
    "Part Gross Weight": ["Part Gross Weight","料號毛重","Gross weight"],
    "Net/Gross Unit": ["Net/Gross Unit","淨毛重單位","單位", "unit", "Net weight unit", "Gross weight unit"]
}

# Flatten category mapping for vectorization
all_headers = [header for headers in CATEGORY_MAP.values() for header in headers]
vectorizer = TfidfVectorizer().fit(all_headers)
header_vectors = vectorizer.transform(all_headers)

def classify_headers_ml(bom_file):
    """Classifies BOM headers into predefined categories using NLP."""
    df = pd.read_excel(bom_file, nrows=1)  # Read only headers
    actual_headers = df.columns.tolist()
    
    matched = {}
    misclassified = {}
    
    for header in actual_headers:
        header_vector = vectorizer.transform([header])
        similarities = cosine_similarity(header_vector, header_vectors)
        best_match_index = similarities.argmax()
        best_match = all_headers[best_match_index]
        
        for category, headers in CATEGORY_MAP.items():
            if best_match in headers:
                matched[header] = category
                break
        else:
            misclassified[header] = "Unknown Category"
    
    not_matched = [category for category in CATEGORY_MAP if category not in matched.values()]
    
    return matched, not_matched, misclassified

def process_bom_files(file_list):
    """Processes multiple BOM files and prints classification results."""
    for file in file_list:
        print(f"\nProcessing {file}...")
        matched, not_matched, misclassified = classify_headers_ml(file)
        
        print("Matched Categories:")
        for header, category in matched.items():
            print(f"  {header} -> {category}")
        
        print("\nCategories Not Matched:")
        for category in not_matched:
            print(f"  {category}")
        
        print("\nMisclassified Headers:")
        for header, category in misclassified.items():
            print(f"  {header} -> {category}")

In [ ]:
#bom_files = [data_path+"bom1.xlsx", data_path+"bom2.xlsx", data_path+"test_bom.xlsx"]
bom_files = [data_path+"case1.xlsx"]
process_bom_files(bom_files)
